In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Flatten, Conv2D

%matplotlib inline

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10

In [3]:
(X_tr, Y_tr),(X_te, Y_te) = cifar10.load_data()
X_tr.shape, Y_tr.shape, X_te.shape, Y_te.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [4]:
from keras.layers import MaxPooling2D, Dropout, Activation
from keras.models import Sequential
from keras.utils import to_categorical

Y_ = np.reshape(Y_tr, (-1))
n_classes = len(set(Y_))



n_classes

10

In [5]:
y_train = to_categorical(Y_tr, n_classes)
y_te = to_categorical(Y_te, n_classes)

In [43]:
def CNN():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_tr.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))          
    model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.5))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))          
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.5))
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))          
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.5))
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))          
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    
    
    # Fully Conected 
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))          
              
    return model          
    

In [44]:
model = CNN()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_47 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_48 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 30, 30, 64)        18496     
_________________________________________________________________
activation_49 (Activation)   (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 28, 28, 64)        36928     
__________

In [45]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [46]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                        height_shift_range=0.08, zoom_range=0.08)

In [47]:
test_gen = ImageDataGenerator()

In [48]:
batch_size = 128


In [49]:
train_generator = gen.flow(X_tr, y_train, batch_size=batch_size)
test_generator = test_gen.flow(X_te, y_te, batch_size=batch_size)

In [50]:
model.fit_generator(train_generator, steps_per_epoch=X_tr.shape[0]//batch_size, epochs=7,
                   validation_data=test_generator, validation_steps=X_te.shape[0]//batch_size)

Epoch 1/7
390/390 [==============================] - 837s 2s/step - loss: 2.1235 - acc: 0.2152 - val_loss: 1.6649 - val_acc: 0.3691
Epoch 2/7
390/390 [==============================] - 848s 2s/step - loss: 1.6527 - acc: 0.3832 - val_loss: 1.3484 - val_acc: 0.5017
Epoch 3/7
390/390 [==============================] - 851s 2s/step - loss: 1.4283 - acc: 0.4830 - val_loss: 1.2207 - val_acc: 0.5582
Epoch 4/7
390/390 [==============================] - 867s 2s/step - loss: 1.2773 - acc: 0.5444 - val_loss: 1.0847 - val_acc: 0.6183
Epoch 5/7
390/390 [==============================] - 855s 2s/step - loss: 1.1808 - acc: 0.5805 - val_loss: 1.0057 - val_acc: 0.6452
Epoch 6/7
390/390 [==============================] - 846s 2s/step - loss: 1.1043 - acc: 0.6149 - val_loss: 0.9585 - val_acc: 0.6670
Epoch 7/7
390/390 [==============================] - 840s 2s/step - loss: 1.0554 - acc: 0.6330 - val_loss: 0.9254 - val_acc: 0.6835


In [51]:
test = model.evaluate(X_te, y_te)
print('Testing results:{}'.format(test[1]))

10000/10000 [==============================] - 55s 6ms/step
Testing results:0.6837
